In [2]:
#equal number in each slice for SIR
from tabulate import tabulate
import numpy as np
from scipy.linalg import solve_triangular
def sir_modified1(X, y, num_slices, K):
    X1 = X - np.mean(X, axis=0)
    #reduced QR decomposition
    Q, R = np.linalg.qr(X1)
    n_samples, n_features = X.shape
    Z = np.sqrt(n_samples) * Q
    
    V_hat1 = np.zeros([X.shape[1], X.shape[1]])
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    Z_sorted = Z[sorted_indices]
    y_sorted = y[sorted_indices]
    X_sorted = X[sorted_indices]
    # X_sorted = X1[sorted_indices]
    
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    slices1 = []
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((Z_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    for j in range(num_slices):
        start_idx1 = j * slice_size
        if j < num_slices - 1:
            end_idx1 = (j + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx1 = n_samples
        slices1.append((X_sorted[start_idx1:end_idx1], y_sorted[start_idx1:end_idx1]))
    
    # Step 3: Compute the means of the predictors within each slice
    Z_means = np.array([np.mean(slice_Z, axis=0) for slice_Z, _ in slices])
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices1])
    # Step 4: Center the predictor means
    Z_centered = Z_means - np.mean(Z_means, axis=0)
    X_centered = X_means - np.mean(X_means, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(Z_centered.T, Z_centered))
    V_hat1 = np.add(V_hat1,ph_hat * np.matmul(X_centered.T, X_centered))
    
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    
    idx = eigenvalues.argsort()[::-1]  # Get indices that would sort eigenvalues in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    edr_est = solve_triangular(R, eigenvectors)
    # edr_est = solve_triangular(np.sqrt(n_samples) * R, eigenvectors)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    edr_est = edr_est[:, K_index]
    # edr_est = edr_est.flatten()
    if edr_est[0][0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    return edr_est.T, V_hat1, V_hat, R

# With whitening for time series

In [19]:
def sir_11(X, y, num_slices, K):
    X = np.linalg.inv(np.cov(X)) @ (X - np.mean(X, axis = 0))
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
        
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    # Step 4: Center the predictor means
    # X_centered = X_means - np.mean(X, axis=0)
    X_centered = (X_means - np.mean(X_means, axis=0)) - np.mean(X, axis=0)
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    return edr_est, V_hat

In [43]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        # X = X -np.mean(X, axis = 0)
        # covariance_matrix = np.cov(X, rowvar=False)
        # eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    #     eigenvectors = np.real(eigenvectors)
    # # Construct the transformation matrix with eigenvalues adjusted
    #     transform_matrix = eigenvectors @ np.diag(1 / np.sqrt(eigenvalues)) @ eigenvectors.T
    # # Apply the transformation to X
    #     X_transformed = X @ transform_matrix
        # _, V_hat = sir_11(X, y[0], H, K)
        # A = np.power(V_hat, -1/2)
        # X = X @ (V_hat)**(-1/2)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_11(X, y[a], H, K)
            # edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            edr[a] = edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                # EDR += np.linalg.inv(np.cov(X.T)) @ edr[0]
            # Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            
            ## For Q>0, I don't know if it's appopriate to multiply np.linalg.inv(np.cov(X.T)) to transform back.
            # edr_est = np.linalg.inv(np.cov(X.T)) @ K_largest_eigenvectors
            # edr_est = np.linalg.inv(V1[0]) @ K_largest_eigenvectors
            edr_est = K_largest_eigenvectors 
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            # hat[q - 1] += edr_est
            hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    print(L)
    print(EDR[0])

# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test(ar_coeff, 2, 3, 10000)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.533996 & 0.845069 & -0.000320675 & -5.29097e-05 & 0.0347705 \\ \hline
 1 & 0.53244  & 0.846015 & -0.000347259 & -8.24337e-06 & 0.0373165 \\ \hline
 2 & 0.532407 & 0.846031 & -0.00035203  & -1.54633e-05 & 0.0373667 \\ \hline
 3 & 0.532404 & 0.84603  & -0.000351838 & -1.56404e-05 & 0.0373694 \\ \hline
 4 & 0.532404 & 0.846029 & -0.000351769 & -1.48809e-05 & 0.0373696 \\ \hline
 5 & 0.532403 & 0.846029 & -0.000351811 & -1.42437e-05 & 0.0373697 \\ \hline
 6 & 0.532403 & 0.846028 & -0.000351814 & -1.42899e-05 & 0.0373697 \\ \hline
 7 & 0.532403 & 0.846028 & -0.000351815 & -1.44554e-05 & 0.0373697 \\ \hline
 8 & 0.532403 & 0.846028 & -0.000351815 & -1.44587e-05 & 0.0373697 \\ \hline
 9 & 0.532403 & 0.846028 & -0.000351816 & -1.44645e-05 & 0.0373697 \\ \hline
\hline \hline
\end{tabular} \hline
0.03477050583893748
[ 5.33995813e-01  8.45068930e-01 -3.20674659e-04 -5.29097416e-05]


In [44]:
Test(ar_coeff, 2, 3, 100)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.271226  & 0.376368  & 0.0189957  & 0.0462102 & 0.0539737 \\ \hline
 1 & 0.124987  & 0.146142  & 0.00127151 & 0.0914678 & 0.188581  \\ \hline
 2 & 0.0848913 & 0.0802231 & 0.00602097 & 0.0633546 & 0.391523  \\ \hline
 3 & 0.0700162 & 0.0637908 & 0.00691467 & 0.0399233 & 0.430924  \\ \hline
 4 & 0.0617555 & 0.0537751 & 0.00582309 & 0.0373718 & 0.481737  \\ \hline
 5 & 0.0572372 & 0.0485935 & 0.00512652 & 0.0370178 & 0.511211  \\ \hline
 6 & 0.0547724 & 0.0458259 & 0.0048593  & 0.0370341 & 0.528562  \\ \hline
 7 & 0.0532974 & 0.0442334 & 0.00466845 & 0.0371053 & 0.538245  \\ \hline
 8 & 0.0524323 & 0.0433275 & 0.00454858 & 0.0371775 & 0.543472  \\ \hline
 9 & 0.0518909 & 0.0427873 & 0.00444713 & 0.0372337 & 0.546098  \\ \hline
\hline \hline
\end{tabular} \hline
0.053973701319496414
[0.27122589 0.37636788 0.01899574 0.04621019]


# Without whitening: multiplying uniform $\Sigma_{xx}^{-1}$

In [35]:
def sir_11(X, y, num_slices, K):
    X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    
    # Step 4: Center the predictor means
    X_centered = X_means - np.mean(X, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    
    return edr_est, V_hat

In [344]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test(ar_coeff, S, a1, a2, n_obs):   #S, n_rep
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    # S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1 
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        # X = X -np.mean(X, axis = 0)
        # covariance_matrix = np.cov(X, rowvar=False)
        # eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    #     eigenvectors = np.real(eigenvectors)
    # # Construct the transformation matrix with eigenvalues adjusted
    #     transform_matrix = eigenvectors @ np.diag(1 / np.sqrt(eigenvalues)) @ eigenvectors.T
    
    # # Apply the transformation to X
    #     X_transformed = X @ transform_matrix
        # _, V_hat = sir_11(X, y[0], H, K)
        # A = np.power(V_hat, -1/2)
        # X = X @ (V_hat)**(-1/2)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_11(X, y[a], H, K)
            edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            # edr[a] = np.power(np.linalg.inv(np.cov(X.T)), 1/2) @ edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                # EDR += np.linalg.inv(np.cov(X.T)) @ edr[0]
            # Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            ## For Q>0, I don't know if it's appopriate to multiply np.linalg.inv(np.cov(X.T)) to transform back.

            edr_est = np.linalg.inv(np.cov(X.T)) @ K_largest_eigenvectors
            
            # edr_est = np.linalg.inv(V1[0]) @ K_largest_eigenvectors
            # edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
            # hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    # array = np.array(hat)
    array = np.vectorize(lambda x: f"{x:.4f}")(hat)
    table = tabulate(array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    # print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # print(L)
    # print(EDR[0])
    return g
    
def EXperi(ar_coeff, S, n_samples, n_end, n_interval):
    # S = 10
    V = []
    # ar_coeff = [0.2, 0.3, 0.5, 0.8]
    nn = n_samples
    
    g = [np.zeros((1, 1)) for _ in range(S)]
    for i in range(S):
        g[i] = Test(ar_coeff, S, 2, 3, nn)[i]
    
    for n in range(nn, n_end, n_interval):
        h = Test(ar_coeff, S, 2, 3, nn)
        for i in range(S):
            g[i] = np.vstack((g[i], h[i].reshape(-1,1)))
    index_array = list(range(len(g)))
    for i in range(S):
        g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))          
    for j in range(1, len(g[1])):    
        sorted_indices = np.argsort([g[i][j] for i in range(S)], axis = 0)
        min_sorted_indices = sorted_indices[0][0]
        V.append(min_sorted_indices)
            
    g = np.concatenate((g, [np.array([[0] + V]).T]), axis = 0)
    
    # g = [np.zeros((1, 1)) for _ in range(S)]
    # for i in range(S):
    #     g[i] = Test(ar_coeff, S, 2, 3, nn)[i]
    # index_array = list(range(len(g)))
    # for i in range(S):
    #     g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))
    # for n in range(nn, nn+n_end, n_interval):
    #     h = Test(ar_coeff, S, 2, 3, nn)
    #     for i in range(S):
    #         g[i] = np.vstack((g[i], h[i].reshape(-1,1)))

            
            # g[i] = np.hstack((g[i], [np.max(np.argsort([g[i][j] for i in range(S)])) for j in range(len(g[1]))] ))
            
    formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
    # array = np.array(g)
    table = tabulate(formatted_array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
# formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
# # array = np.array(g)
# table = tabulate(formatted_array, tablefmt='latex_raw')

# # Split the table into lines
# lines = table.split('\n')

# # Insert \hline after each row
# latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])

In [339]:
V = []
# ar_coeff = [0.2, 0.3, 0.5, 0.8]
nn = 10

g = [np.zeros((1, 1)) for _ in range(S)]
for i in range(S):
    g[i] = Test(ar_coeff, S, 2, 3, nn)[i]

for n in range(nn, nn+10, 2):
    h = Test(ar_coeff, S, 2, 3, nn)
    for i in range(S):
        g[i] = np.vstack((g[i], h[i].reshape(-1,1)))
index_array = list(range(len(g)))
for i in range(S):
    g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))          
for j in range(1, len(g[1])):    
    sorted_indices = np.argsort([g[i][j] for i in range(S)], axis = 0)
    min_sorted_indices = sorted_indices[0][0]
    V.append(min_sorted_indices)
        
g = np.concatenate((g, [np.array([[0] + V]).T]), axis = 0)

In [338]:
[g[i][2] for i in range(S)]

[array([0.59624592]),
 array([0.48616811]),
 array([1.18246793]),
 array([2.90494606]),
 array([2.9751451]),
 array([3.15473019]),
 array([4.24782505]),
 array([3.91007996]),
 array([3.98200932]),
 array([4.03918018])]

In [340]:
V

[1, 1, 0, 1, 1, 0]

In [ ]:
# def EXperi(ar_coeff, S, n_samples, n_end, n_interval)

In [346]:
ar_coeff = [0.1, 0.1, 0.5, 0.8]
EXperi(ar_coeff, 20, 50, 100, 10)

\begin{tabular}{rrrrrrr}
\hline
  0 &  0.0177 & 0.0021 &  0.0141 & 0.0035 & 0.0119 & 0.0163 \\ \hline
  1 &  0.0106 & 0.0016 &  0.0104 & 0.0017 & 0.0175 & 0.0175 \\ \hline
  2 &  0.0082 & 0.008  &  0.0078 & 0.0007 & 0.0213 & 0.0186 \\ \hline
  3 &  0.0119 & 0.0123 &  0.0072 & 0.0021 & 0.0231 & 0.0203 \\ \hline
  4 &  0.0007 & 0.0059 &  0.0071 & 0.0034 & 0.0246 & 0.0209 \\ \hline
  5 &  0.0114 & 0.0063 &  0.0069 & 0.0207 & 0.0278 & 0.0214 \\ \hline
  6 &  0.0259 & 0.0081 &  0.0063 & 0.0243 & 0.0278 & 0.0217 \\ \hline
  7 &  0.0246 & 0.01   &  0.0062 & 0.0249 & 0.0277 & 0.0218 \\ \hline
  8 &  0.0073 & 0.0001 &  0.0061 & 0.025  & 0.0276 & 0.0219 \\ \hline
  9 &  0.0171 & 0.0001 &  0.0061 & 0.0248 & 0.0276 & 0.0219 \\ \hline
 10 &  0.005  & 0.0002 &  0.0061 & 0.0248 & 0.0275 & 0.022  \\ \hline
 11 &  0.0152 & 0.0002 &  0.0061 & 0.0248 & 0.0274 & 0.022  \\ \hline
 12 &  0.0099 & 0.0003 &  0.0061 & 0.0248 & 0.0274 & 0.022  \\ \hline
 13 &  0.0001 & 0.0003 &  0.0061 & 0.0248 & 0.0274 & 0.022

In [306]:
ar_coeff = [0.1, 0.1, 0.5, 0.8]
EXperi(ar_coeff, 10, 50, 50, 10)

\begin{tabular}{rrrrrrr}
\hline
 0 & 0.0027 & 0.0085 & 0.0027 & 0.0034 & 0.0096 & 0.0136 \\ \hline
 1 & 0.0161 & 0.0072 & 0.0042 & 0.0012 & 0.0069 & 0.0151 \\ \hline
 2 & 0.019  & 0.0066 & 0.0087 & 0.0001 & 0.0044 & 0.0152 \\ \hline
 3 & 0.0216 & 0.0086 & 0.01   & 0.0006 & 0.0036 & 0.0164 \\ \hline
 4 & 0.0234 & 0.0013 & 0.0106 & 0.0048 & 0.0035 & 0.0176 \\ \hline
 5 & 0.013  & 0.0014 & 0.0105 & 0.0051 & 0.0032 & 0.0182 \\ \hline
 6 & 0.0121 & 0.0013 & 0.0103 & 0.0051 & 0.0033 & 0.0186 \\ \hline
 7 & 0.003  & 0.0013 & 0.0103 & 0.0051 & 0.0033 & 0.0189 \\ \hline
 8 & 0.0158 & 0.0013 & 0.0103 & 0.005  & 0.0032 & 0.019  \\ \hline
 9 & 0.0135 & 0.0012 & 0.0103 & 0.005  & 0.0032 & 0.0191 \\ \hline
 0 & 0      & 8      & 0      & 3      & 9      & 0      \\ \hline
\hline \hline
\end{tabular} \hline


In [185]:
ar_coeff = [0.1, 0.5, 0.5, 0.8]
EXperi(ar_coeff, 500)

\begin{tabular}{rrrrrrrr}
\hline
 0 & 0.005  & 0.0036 & 0.004  & 0.0057 & 0.0022 & 0.0023 & 0.0032 \\ \hline
 1 & 0.0291 & 0.029  & 0.0284 & 0.0383 & 0.0344 & 0.0304 & 0.0361 \\ \hline
 2 & 0.0364 & 0.0315 & 0.0309 & 0.0406 & 0.0367 & 0.0327 & 0.0386 \\ \hline
 3 & 0.0374 & 0.0317 & 0.0312 & 0.0408 & 0.037  & 0.0329 & 0.0389 \\ \hline
 4 & 0.0403 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
 5 & 0.0361 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
 6 & 0.0392 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
 7 & 0.0348 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
 8 & 0.0359 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
 9 & 0.0396 & 0.0318 & 0.0312 & 0.0409 & 0.037  & 0.033  & 0.0389 \\ \hline
\hline \hline
\end{tabular} \hline


In [187]:
ar_coeff = [0.1, 0.2, 0.8, 0.8]
EXperi(ar_coeff, 50)

\begin{tabular}{rrrrrrrr}
\hline
 0 & 0.0236 & 0.0192 & 0.0089 & 0.0109 & 0.0067 & 0.0087 & 0.0184 \\ \hline
 1 & 0.0037 & 0.0138 & 0.0122 & 0.0188 & 0.0008 & 0.0114 & 0.0253 \\ \hline
 2 & 0.0193 & 0.0139 & 0.0064 & 0.0342 & 0.0312 & 0.0113 & 0.0256 \\ \hline
 3 & 0.0229 & 0.0138 & 0.0031 & 0.0427 & 0.0443 & 0.0278 & 0.0195 \\ \hline
 4 & 0.0216 & 0.0129 & 0.0083 & 0.0449 & 0.0488 & 0.046  & 0.0191 \\ \hline
 5 & 0.0209 & 0.0128 & 0.0086 & 0.0461 & 0.0519 & 0.0504 & 0.0188 \\ \hline
 6 & 0.0525 & 0.023  & 0.0085 & 0.0467 & 0.0541 & 0.0519 & 0.0184 \\ \hline
 7 & 0.0002 & 0.0222 & 0.0085 & 0.0592 & 0.0556 & 0.053  & 0.0037 \\ \hline
 8 & 0.0108 & 0.0218 & 0.0083 & 0.0592 & 0.0567 & 0.0542 & 0.003  \\ \hline
 9 & 0.0394 & 0.0213 & 0.0083 & 0.0594 & 0.0572 & 0.0559 & 0.0026 \\ \hline
\hline \hline
\end{tabular} \hline


In [189]:
ar_coeff = [0.1, 0.6, 0.8, 0.8]
EXperi(ar_coeff, 50)

\begin{tabular}{rrrrrrrr}
\hline
 0 & 0.0225 & 0.0233 & 0.003  & 0.0251 & 0.0173 & 0.0009 & 0.0194 \\ \hline
 1 & 0.1029 & 0.0965 & 0.0766 & 0.107  & 0.076  & 0.0904 & 0.0967 \\ \hline
 2 & 0.1185 & 0.1119 & 0.0945 & 0.1226 & 0.081  & 0.1038 & 0.1068 \\ \hline
 3 & 0.1156 & 0.1158 & 0.1007 & 0.1188 & 0.0814 & 0.1055 & 0.1082 \\ \hline
 4 & 0.1287 & 0.1157 & 0.0976 & 0.1192 & 0.0813 & 0.1055 & 0.1105 \\ \hline
 5 & 0.0948 & 0.1157 & 0.0984 & 0.1189 & 0.0807 & 0.1055 & 0.1109 \\ \hline
 6 & 0.1011 & 0.1158 & 0.0988 & 0.1184 & 0.0807 & 0.1052 & 0.1107 \\ \hline
 7 & 0.1254 & 0.1159 & 0.099  & 0.1062 & 0.0798 & 0.1049 & 0.1105 \\ \hline
 8 & 0.1364 & 0.1055 & 0.099  & 0.106  & 0.0793 & 0.1044 & 0.1102 \\ \hline
 9 & 0.1099 & 0.1053 & 0.099  & 0.1057 & 0.0791 & 0.1042 & 0.1102 \\ \hline
\hline \hline
\end{tabular} \hline


In [190]:
ar_coeff = [0.2, 0.7, 0.8, 0.8]
EXperi(ar_coeff, 50)

\begin{tabular}{rrrrrrrr}
\hline
 0 & 0.026  & 0.0261 & 0.0442 & 0.0219 & 0.0087 & 0.0277 & 0.0389 \\ \hline
 1 & 0.1468 & 0.1471 & 0.1453 & 0.1308 & 0.112  & 0.1394 & 0.1611 \\ \hline
 2 & 0.1715 & 0.1729 & 0.1623 & 0.1559 & 0.1374 & 0.1641 & 0.1907 \\ \hline
 3 & 0.1931 & 0.1825 & 0.1667 & 0.1636 & 0.144  & 0.1721 & 0.1905 \\ \hline
 4 & 0.1944 & 0.175  & 0.1692 & 0.1673 & 0.1459 & 0.1745 & 0.1945 \\ \hline
 5 & 0.1638 & 0.1759 & 0.1717 & 0.1576 & 0.1469 & 0.1749 & 0.1967 \\ \hline
 6 & 0.195  & 0.166  & 0.1729 & 0.158  & 0.147  & 0.1748 & 0.1976 \\ \hline
 7 & 0.1829 & 0.1661 & 0.1736 & 0.1582 & 0.147  & 0.1749 & 0.1981 \\ \hline
 8 & 0.184  & 0.1661 & 0.1738 & 0.1583 & 0.147  & 0.1749 & 0.1983 \\ \hline
 9 & 0.2148 & 0.166  & 0.1739 & 0.1584 & 0.147  & 0.1749 & 0.1983 \\ \hline
\hline \hline
\end{tabular} \hline


# Estimate Cov(E(X|y)), multiply $\phi^{-1}$ and average

In [40]:
def sir_11(X, y, num_slices, K):
    X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    
    # Step 4: Center the predictor means
    X_centered = X_means - np.mean(X, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    
    return edr_est, V_hat

In [192]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test1(ar_coeff, a1, a2, n_obs, S, n_rep):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    X1 = [[np.zeros((num_N, n_obs)) for i in range(num_N - 1)] for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S + 1)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1 
    n = n_rep
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    Hat = np.zeros((P, 1))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
            X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1,1) for i in range(num_N - 1)], axis = 1)
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(P)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_11(X, y[a], H, K)
            edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                
        for q in range(0, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            Q3 = np.linalg.inv(np.cov(X1[q].T)) @ V1[q] @ np.linalg.inv(np.cov(X1[q].T))   #(phi[j] ** a)
            # Q3 = np.linalg.inv(np.cov(X.T)) @ V1[q] @ np.linalg.inv(np.cov(X.T))
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            
            edr_est = np.multiply(np.power(ar_coeff, -q), K_largest_eigenvectors.flatten())
            
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q] += edr_est.reshape(-1, 1)
            # hat[q - 1] += np.real(edr_est)
        n1 += 1
        
        Hat += sum(hat[i] for i in range(0, S + 1))
        
    print(Hat)

In [84]:
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test1(ar_coeff, 2, 3, 100, 10, 10000)

[[ 183.04482779]
 [ 232.79206203]
 [ 444.94668317]
 [2255.38881703]]


In [124]:
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test1(ar_coeff, 2, 3, 100, 10, 1000)

[[ 4.31144061e+02]
 [ 4.72170178e+01]
 [ 9.27838411e-01]
 [-2.14612349e-01]]


In [193]:
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test1(ar_coeff, 2, 3, 100, 1, 1000)

[[569496.87765692]
 [532752.73381324]
 [  -884.85593776]
 [  -813.2256108 ]]


In [157]:
import numpy as np
from tabulate import tabulate
num_N = 5
n_obs = 1000
S = 10
n_rep = 10
a1 = 2
a2 = 3
noise = np.zeros((num_N, n_obs+S))
H = 5
P = 4
K = 1
y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
X1 = [[np.zeros((num_N, n_obs)) for i in range(num_N - 1)] for i in range(S+1)]
hat = [np.zeros((P, 1)) for _ in range(S + 1)]
g = np.zeros((S, 1))
n1 = 0
l = 1 
n = n_rep
edr = np.zeros((S+1, P))
EDR = np.zeros((1, P))
while n1 < n:
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs+S))
    for t in range(0, n_obs+S):
        ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
        ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
        ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
        ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
        ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
    for a in range(0, S+1):
        y[a] = ar_series[4][a:n_obs+a]
        X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1,1) for i in range(num_N - 1)], axis = 1)
    X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
    V1 = []
    Q2 = []
    for a in range(0, S + 1):
        edr_part, M = sir_11(X, y[a], H, K)
        edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
        V1.append(M)
        if a == 0:
            if edr[0][0]<0:
                edr[0] = -edr[0]
            edr[0] = edr[0]/np.linalg.norm(edr[0])
            EDR += edr[0]
            
    for q in range(0, S + 1):
        Q3 = np.zeros((P, P))
        phi = ar_coeff
        Q3 = np.linalg.inv(np.cov(X.T)) @ V1[q] @ np.linalg.inv(np.cov(X.T))   #(phi[j] ** a)
        # Q3 = np.linalg.inv(np.cov(X1[q].T)) @ V1[q] @ np.linalg.inv(np.cov(X1[q].T))
        eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
        # edr_est = solve_triangular(Q2[0], eigenvectors1)
        K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
        # K_largest_eigenvectors = edr_est[:, K_index]
        K_largest_eigenvectors = eigenvectors1[:, K_index]

        edr_est = np.multiply(np.power(ar_coeff, -q), K_largest_eigenvectors.flatten())      
        if edr_est[0] < 0:
            edr_est = -edr_est
        edr_est = edr_est / np.linalg.norm(edr_est)
        hat[q] += edr_est.reshape(-1, 1)
        # hat[q - 1] += np.real(edr_est)
    n1 += 1

In [205]:
Q = np.zeros((P, P))
phi = ar_coeff
m = 0
Q = np.linalg.inv(np.cov(X.T)) @ V1[m] @ np.linalg.inv(np.cov(X.T))   #(phi[j] ** a)
# Q = np.linalg.inv(np.cov(X1[m].T)) @ V1[m] @ np.linalg.inv(np.cov(X1[m].T))
eigenvalues, eigenvectors = np.linalg.eig(Q3)
# edr_est = solve_triangular(Q2[0], eigenvectors1)
K_index = np.argpartition(np.abs(eigenvalues), P - K) >= P - K
# K_largest_eigenvectors = edr_est[:, K_index]
K_largest_eigenvectors = eigenvectors[:, K_index]

edr_est = np.multiply(np.power(ar_coeff, -m), K_largest_eigenvectors.flatten())

In [202]:
K_largest_eigenvectors.flatten()

array([-0.3816451 ,  0.06614549,  0.21743479, -0.89593186])

In [206]:
edr_est

array([-0.3816451 ,  0.06614549,  0.21743479, -0.89593186])

# Without whitening: multiplying seperate $\Sigma_{(x-q)(x-q)}^{-1}$

In [18]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test1(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 50
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        
        # X = X -np.mean(X, axis = 0)
        # covariance_matrix = np.cov(X, rowvar=False)
        # eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    #     eigenvectors = np.real(eigenvectors)
    # # Construct the transformation matrix with eigenvalues adjusted
    #     transform_matrix = eigenvectors @ np.diag(1 / np.sqrt(eigenvalues)) @ eigenvectors.T
    
    # # Apply the transformation to X
    #     X_transformed = X @ transform_matrix
        # _, V_hat = sir_11(X, y[0], H, K)
        # A = np.power(V_hat, -1/2)
        # X = X @ (V_hat)**(-1/2)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_11(X, y[a], H, K)
            edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                # EDR += np.linalg.inv(np.cov(X.T)) @ edr[0]
            # Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X[ : X.shape[0] - q].T)) @ V1[a] @ np.linalg.inv(np.cov(X[: X.shape[0] - q].T)))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]

            ## For Q>0, I don't know if it's appopriate to multiply np.linalg.inv(np.cov(X.T)) to transform back.

            edr_est = K_largest_eigenvectors
            # edr_est = np.linalg.inv(np.cov(X.T)) @ K_largest_eigenvectors
            # edr_est = np.linalg.inv(V1[0]) @ K_largest_eigenvectors
            # edr_est = K_largest_eigenvectors 
            
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
            # hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    print(L)
    print(EDR[0])

# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test1(ar_coeff, 2, 3, 80)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.593387 & -0.200007  & 0.0382008 & -0.0312047   & 3.6335  \\ \hline
 1 & 0.433993 & -0.167195  & 0.122721  &  0.000265494 & 3.2624  \\ \hline
 2 & 0.365873 & -0.0963971 & 0.110853  & -0.0414165   & 4.46214 \\ \hline
 3 & 0.330933 & -0.0836644 & 0.129064  & -0.0546248   & 4.62215 \\ \hline
 4 & 0.315857 & -0.0742031 & 0.118406  & -0.0346726   & 4.92332 \\ \hline
 5 & 0.304829 & -0.0627978 & 0.0697248 & -0.0116148   & 5.5208  \\ \hline
 6 & 0.276944 & -0.0377743 & 0.0667104 & -0.0429464   & 7.99822 \\ \hline
 7 & 0.268241 & -0.0425712 & 0.089962  &  0.0109939   & 6.96765 \\ \hline
 8 & 0.27968  & -0.055209  & 0.0788766 &  0.0017535   & 5.73252 \\ \hline
 9 & 0.28359  & -0.045081  & 0.0875949 & -0.00793149  & 6.95734 \\ \hline
\hline \hline
\end{tabular} \hline
3.9199833423703767
[ 0.20436142 -0.06281633 -0.05842166 -0.02172449]


In [5]:
Test1(ar_coeff, 2, 3, 1000)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.556099 & 0.831002 & -0.000318917 & 0.000525384 & 0.0025236  \\ \hline
 1 & 0.553594 & 0.832668 & -0.000299849 & 0.000443954 & 0.0018238  \\ \hline
 2 & 0.553392 & 0.832799 & -0.000321327 & 0.000265707 & 0.0021706  \\ \hline
 3 & 0.553518 & 0.832714 & -0.000384392 & 0.000216875 & 0.00195063 \\ \hline
 4 & 0.553466 & 0.832748 & -0.000364973 & 0.000210582 & 0.0020399  \\ \hline
 5 & 0.553672 & 0.832608 & -0.000386296 & 0.000180199 & 0.00168187 \\ \hline
 6 & 0.553628 & 0.832634 & -0.000353458 & 0.00016512  & 0.00175555 \\ \hline
 7 & 0.553672 & 0.832604 & -0.000341669 & 0.000104764 & 0.00167743 \\ \hline
 8 & 0.553781 & 0.832529 & -0.000430381 & 6.00476e-05 & 0.00148733 \\ \hline
 9 & 0.553902 & 0.832444 & -0.000523843 & 0.000169128 & 0.00127447 \\ \hline
\hline \hline
\end{tabular} \hline
0.002346436309913491
[ 5.53287446e-01  8.32862558e-01 -1.48464863e-04  9.14297656e-05]


In [6]:
Test1(ar_coeff, 2, 3, 100)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.551886 & 0.784502 &  0.00553951  &  0.00661992 & 0.0368188  \\ \hline
 1 & 0.522316 & 0.779048 & -0.00098017  &  0.00837238 & 0.00378759 \\ \hline
 2 & 0.516206 & 0.749346 &  0.00602649  &  0.0084525  & 0.0222086  \\ \hline
 3 & 0.5039   & 0.71601  &  0.0221853   & -0.00436081 & 0.0370952  \\ \hline
 4 & 0.489379 & 0.703581 &  0.0206358   & -0.0103419  & 0.0288874  \\ \hline
 5 & 0.483794 & 0.685206 &  0.000415459 & -0.010076   & 0.0393893  \\ \hline
 6 & 0.470539 & 0.656758 &  0.00292458  &  0.00798477 & 0.049791   \\ \hline
 7 & 0.450217 & 0.628528 &  0.0118395   & -0.0362715  & 0.0496371  \\ \hline
 8 & 0.455805 & 0.583056 &  0.0353442   &  0.00391622 & 0.115085   \\ \hline
 9 & 0.45207  & 0.591922 &  0.0150978   & -0.0203426  & 0.0970654  \\ \hline
\hline \hline
\end{tabular} \hline
0.0835884502324854
[ 0.38181995  0.50892016 -0.01455863 -0.0018383 ]


In [7]:
Test1(ar_coeff, 2, 3, 10)

C:\Users\yhuang73\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\yhuang73\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


LinAlgError: Array must not contain infs or NaNs

In [8]:
Test1(ar_coeff, 2, 3, 50)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.538026 &  0.0129157  &  0.0579676   &  0.0194087  &   40.9901  \\ \hline
 1 & 0.413918 & -0.00538281 &  0.00464111  &  0.0249488  &   77.5629  \\ \hline
 2 & 0.382507 & -0.00618278 & -0.00155372  &  0.0696794  &   62.5331  \\ \hline
 3 & 0.352734 & -0.0417019  &  0.0145337   &  0.0500557  &    9.12513 \\ \hline
 4 & 0.321082 & -0.0264536  &  0.0406214   &  0.0104744  &   12.8042  \\ \hline
 5 & 0.327509 & -0.0105641  &  0.00301076  & -0.00200243 &   31.6686  \\ \hline
 6 & 0.321886 &  0.0134612  &  0.000872697 &  0.00114963 &   23.2454  \\ \hline
 7 & 0.288034 & -0.017865   & -0.0345217   & -0.00628745 &   16.7895  \\ \hline
 8 & 0.281621 &  0.00023571 &  0.030861    & -0.0938845  & 1194.11    \\ \hline
 9 & 0.271379 & -0.0010959  &  0.0293359   & -0.0545872  &  248.298   \\ \hline
\hline \hline
\end{tabular} \hline
8.641277273918536
[ 0.18772292  0.02016803 -0.02099418 -0.01092285]


# Without whitening: Using the remark, multiplying uniform $\Sigma_{xx}^{-1}$

In [20]:
def sir_12(X, y, num_slices, K):
    # X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    
    # Step 4: Center the predictor means
    X_centered = (X_means - np.mean(X_means, axis=0)) - np.mean(X, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    
    return edr_est, V_hat

In [84]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 50
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        # X = X -np.mean(X, axis = 0)
        # covariance_matrix = np.cov(X, rowvar=False)
        # eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    #     eigenvectors = np.real(eigenvectors)
    # # Construct the transformation matrix with eigenvalues adjusted
    #     transform_matrix = eigenvectors @ np.diag(1 / np.sqrt(eigenvalues)) @ eigenvectors.T
    
    # # Apply the transformation to X
    #     X_transformed = X @ transform_matrix
        # _, V_hat = sir_11(X, y[0], H, K)
        # A = np.power(V_hat, -1/2)
        # X = X @ (V_hat)**(-1/2)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_12(X, y[a], H, K)
            edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                # EDR += np.linalg.inv(np.cov(X.T)) @ edr[0]
            # Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]

            
            ## For Q>0, I don't know if it's appopriate to multiply np.linalg.inv(np.cov(X.T)) to transform back.

            
            edr_est = np.linalg.inv(np.cov(X.T)) @ K_largest_eigenvectors
            # edr_est = np.linalg.inv(V1[0]) @ K_largest_eigenvectors
            # edr_est = K_largest_eigenvectors 
            
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
            # hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    print(L)
    print(EDR[0])

# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test(ar_coeff, 2, 3, 10000)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.554682 & 0.832051 & -0.000421317 & 0.000186286 & 2.20273e-05 \\ \hline
 1 & 0.553119 & 0.83309  & -0.000439969 & 0.00016243  & 0.00272982  \\ \hline
 2 & 0.553089 & 0.83311  & -0.000441105 & 0.000154199 & 0.00278143  \\ \hline
 3 & 0.553088 & 0.833111 & -0.000441671 & 0.000153945 & 0.00278405  \\ \hline
 4 & 0.553088 & 0.833111 & -0.000441617 & 0.000153888 & 0.00278423  \\ \hline
 5 & 0.553088 & 0.833111 & -0.000441578 & 0.000153635 & 0.00278421  \\ \hline
 6 & 0.553088 & 0.833111 & -0.000441571 & 0.000153486 & 0.00278419  \\ \hline
 7 & 0.553088 & 0.833111 & -0.000441565 & 0.000153339 & 0.00278417  \\ \hline
 8 & 0.553088 & 0.833111 & -0.00044156  & 0.000153171 & 0.00278416  \\ \hline
 9 & 0.553088 & 0.833111 & -0.000441558 & 0.000153149 & 0.00278415  \\ \hline
\hline \hline
\end{tabular} \hline
2.202731067624253e-05
[ 5.54682402e-01  8.32051094e-01 -4.21316764e-04  1.86285851e-04]


In [85]:
Test(ar_coeff, 2, 3, 100)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.309112  &  0.411598  & -0.00390617 & -0.0802902  & 0.0843382 \\ \hline
 1 & 0.138667  &  0.111118  &  0.046077   &  0.0530215  & 0.581254  \\ \hline
 2 & 0.103438  &  0.0156685 &  0.0247304  &  0.0786053  & 5.93502   \\ \hline
 3 & 0.0989856 & -0.0170933 &  0.0310242  &  0.00134154 & 6.45758   \\ \hline
 4 & 0.098517  & -0.024111  &  0.0213662  &  0.0408205  & 4.75264   \\ \hline
 5 & 0.0994216 & -0.031697  &  0.0201814  &  0.0407384  & 3.80329   \\ \hline
 6 & 0.100216  & -0.0329793 &  0.0244274  &  0.0803575  & 3.70544   \\ \hline
 7 & 0.100757  & -0.0333101 &  0.0244933  &  0.0802991  & 3.6915    \\ \hline
 8 & 0.101101  & -0.0353179 &  0.0240775  &  0.120005   & 3.52928   \\ \hline
 9 & 0.10139   & -0.0353391 &  0.0240672  &  0.119992   & 3.53571   \\ \hline
\hline \hline
\end{tabular} \hline
0.08433823204715973
[ 0.30911181  0.41159759 -0.00390617 -0.08029018]


In [82]:
Test(ar_coeff, 2, 3, 500)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.550501 & 0.83389  & 0.00305112 &  0.00186006 & 0.00650678 \\ \hline
 1 & 0.53796  & 0.821928 & 0.0105533  & -0.0166877  & 0.0121569  \\ \hline
 2 & 0.51687  & 0.791945 & 0.01045    & -0.0402244  & 0.0140085  \\ \hline
 3 & 0.509471 & 0.784104 & 0.0139635  & -0.0437469  & 0.0169175  \\ \hline
 4 & 0.489092 & 0.752235 & 0.00639513 & -0.0163641  & 0.0164819  \\ \hline
 5 & 0.479128 & 0.735576 & 0.00696986 & -0.00597958 & 0.0153031  \\ \hline
 6 & 0.474131 & 0.727604 & 0.00768256 & -0.00087513 & 0.0150338  \\ \hline
 7 & 0.47105  & 0.722745 & 0.00814339 &  0.00297938 & 0.0149155  \\ \hline
 8 & 0.469051 & 0.719649 & 0.00846258 &  0.00525785 & 0.0148896  \\ \hline
 9 & 0.467886 & 0.717947 & 0.00866254 &  0.00659333 & 0.0149668  \\ \hline
\hline \hline
\end{tabular} \hline
0.006506781450693722
[0.55050052 0.83388969 0.00305112 0.00186006]


In [88]:
Test(ar_coeff, 2, 3, 80)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.17341   & -0.0596272  & -0.0142863   &  0.0767942 &  3.57491 \\ \hline
 1 & 0.103034  & -0.00214893 & -0.00794909  &  0.0997632 & 48.6132  \\ \hline
 2 & 0.0956537 &  0.0257416  &  0.0068056   &  0.0563227 &  3.04925 \\ \hline
 3 & 0.0978862 &  0.0376219  &  0.00963144  & -0.0033661 &  1.93517 \\ \hline
 4 & 0.101353  &  0.0332455  &  0.00525271  & -0.0239397 &  2.38195 \\ \hline
 5 & 0.103515  &  0.0332268  &  0.00316358  & -0.0245252 &  2.44874 \\ \hline
 6 & 0.104691  &  0.0332793  &  0.00204471  & -0.0248031 &  2.47915 \\ \hline
 7 & 0.105388  &  0.0308755  & -0.000658166 & -0.0447117 &  2.74667 \\ \hline
 8 & 0.105831  &  0.0309131  & -0.00106419  & -0.0448128 &  2.75683 \\ \hline
 9 & 0.10611   &  0.0309351  & -0.00130859  & -0.0448806 &  2.76342 \\ \hline
\hline \hline
\end{tabular} \hline
3.5749057348304305
[ 0.17341027 -0.05962724 -0.01428628  0.07679424]


In [86]:
Test(ar_coeff, 2, 3, 20)

LinAlgError: Array must not contain infs or NaNs

# Without whitening: Using the remark, multiplying uniform $\Sigma_{(x-q)(x-q)}^{-1}$

In [ ]:
def sir_12(X, y, num_slices, K):
    # X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    
    # Step 4: Center the predictor means
    X_centered = (X_means - np.mean(X_means, axis=0)) - np.mean(X, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    
    return edr_est, V_hat

In [23]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def Test2(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        # X = X -np.mean(X, axis = 0)
        # covariance_matrix = np.cov(X, rowvar=False)
        # eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    #     eigenvectors = np.real(eigenvectors)
    # # Construct the transformation matrix with eigenvalues adjusted
    #     transform_matrix = eigenvectors @ np.diag(1 / np.sqrt(eigenvalues)) @ eigenvectors.T
    
    # # Apply the transformation to X
    #     X_transformed = X @ transform_matrix
        # _, V_hat = sir_11(X, y[0], H, K)
        # A = np.power(V_hat, -1/2)
        # X = X @ (V_hat)**(-1/2)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr_part, M = sir_12(X, y[a], H, K)
            edr[a] = np.linalg.inv(np.cov(X.T)) @ edr_part.flatten()
            V1.append(M)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
                # EDR += np.linalg.inv(np.cov(X.T)) @ edr[0]
            # Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X[:X.shape[0]-q].T)) @ V1[a] @ np.linalg.inv(np.cov(X[:X.shape[0]-q].T)))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]

            
            ## For Q>0, I don't know if it's appopriate to multiply np.linalg.inv(np.cov(X.T)) to transform back.

            
            edr_est = K_largest_eigenvectors
            # edr_est = np.linalg.inv(V1[0]) @ K_largest_eigenvectors
            # edr_est = K_largest_eigenvectors 
            
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
            # hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')
    
    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    print(L)
    print(EDR[0])

In [24]:
# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
Test2(ar_coeff, 2, 3, 100)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.54218  & 0.765786 & -0.0018391  &  0.00136761 & 0.0413374 \\ \hline
 1 & 0.485398 & 0.668555 &  0.00610551 &  0.013447   & 0.0593743 \\ \hline
 2 & 0.448478 & 0.644463 & -0.00906163 &  0.00192498 & 0.0292271 \\ \hline
 3 & 0.425215 & 0.59763  & -0.0275903  & -0.020329   & 0.0448361 \\ \hline
 4 & 0.409769 & 0.57083  & -0.0144202  &  0.0005827  & 0.0511818 \\ \hline
 5 & 0.398951 & 0.537193 & -0.0165699  & -0.0153067  & 0.0759924 \\ \hline
 6 & 0.38841  & 0.545203 & -0.032914   & -0.0302815  & 0.0457473 \\ \hline
 7 & 0.388799 & 0.512585 & -0.0192266  & -0.0112626  & 0.0918398 \\ \hline
 8 & 0.375265 & 0.507318 & -0.0192327  & -0.00767221 & 0.0730379 \\ \hline
 9 & 0.358338 & 0.477938 & -0.00953313 & -0.0161782  & 0.083091  \\ \hline
\hline \hline
\end{tabular} \hline
0.1094852165938277
[ 0.33154161  0.42716074 -0.03310437  0.02474221]


In [ ]:
Test2(ar_coeff, 2, 3, 100)

In the paper, the remark said "just the eigenvectors for the eigenvalue decom-
 position ofnew V_hat1 with respectsigma_xxt, it is the generalized eigenvalue problem."

# first term by Z, following terms by X

In [ ]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def test1(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 25
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1

    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')

    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)
    # print(tabulate(array, tablefmt='latex'))
    # print(g)
    print(L)
    print(EDR[0])
    print(edr_est)
# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
test1(ar_coeff, 2, 3, 10000)

# Take sample covariance matrix by whitened Z

In [38]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test1(ar_coeff, a1, a2, n_obs, S):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 25
    # S = 20
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1

    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')

    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)
    # print(tabulate(array, tablefmt='latex'))
    # print(g)
    print(L)
    print(EDR[0])
    print(edr_est)

In [30]:
ar_coeff = [0.2, 0.3, 0.5, 0.8]
test1(ar_coeff, 2, 3, 10000)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.554628 & 0.832074 & -0.000102234 & 0.00055957  & 0.000105724 \\ \hline
  1 & 0.553198 & 0.833025 & -0.000124745 & 0.000542629 & 0.00258346  \\ \hline
  2 & 0.553181 & 0.833036 & -0.000126783 & 0.000540018 & 0.00261238  \\ \hline
  3 & 0.553181 & 0.833036 & -0.000126837 & 0.000539695 & 0.00261285  \\ \hline
  4 & 0.553181 & 0.833036 & -0.000126849 & 0.000539648 & 0.00261287  \\ \hline
  5 & 0.553181 & 0.833036 & -0.000126849 & 0.000539663 & 0.00261287  \\ \hline
  6 & 0.553181 & 0.833036 & -0.000126849 & 0.000539677 & 0.00261287  \\ \hline
  7 & 0.553181 & 0.833036 & -0.000126849 & 0.000539685 & 0.00261287  \\ \hline
  8 & 0.553181 & 0.833036 & -0.000126849 & 0.000539693 & 0.00261287  \\ \hline
  9 & 0.553181 & 0.833036 & -0.000126849 & 0.000539697 & 0.00261287  \\ \hline
 10 & 0.553181 & 0.833036 & -0.000126849 & 0.000539699 & 0.00261287  \\ \hline
 11 & 0.553181 & 0.833036 & -0.000126849 & 0.000539701 & 0.00261287  \\ \hline
 12 & 0.553181 & 0.83

In [31]:
test1(ar_coeff, 2, 3, 80)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.552821 & 0.828806 & 0.00760208 & 0.00024703  & 0.000342341 \\ \hline
  1 & 0.550644 & 0.830132 & 0.00663265 & 0.000102364 & 0.0033454   \\ \hline
  2 & 0.550427 & 0.830197 & 0.00651015 & 0.000112878 & 0.00365886  \\ \hline
  3 & 0.550373 & 0.83017  & 0.00648169 & 0.00013442  & 0.00370209  \\ \hline
  4 & 0.550347 & 0.830148 & 0.00648132 & 0.000174597 & 0.00371651  \\ \hline
  5 & 0.550334 & 0.830134 & 0.00647843 & 0.000165339 & 0.00372123  \\ \hline
  6 & 0.550325 & 0.830125 & 0.00647681 & 0.000161981 & 0.0037245   \\ \hline
  7 & 0.550319 & 0.830119 & 0.00647283 & 0.000171757 & 0.00372719  \\ \hline
  8 & 0.550315 & 0.830115 & 0.00647004 & 0.00017017  & 0.00372892  \\ \hline
  9 & 0.550312 & 0.830113 & 0.00646886 & 0.000167454 & 0.00372991  \\ \hline
 10 & 0.550311 & 0.830111 & 0.00646851 & 0.000168872 & 0.00373046  \\ \hline
 11 & 0.55031  & 0.830111 & 0.0064683  & 0.000169395 & 0.00373072  \\ \hline
 12 & 0.55031  & 0.83011  & 0.00646812 & 0.00

Compare the results where it's without the whitening step, the method with the QR whitening step seems to have higher accuracy for small number of samples.

In [32]:
test1(ar_coeff, 2, 3, 40)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.547179 & 0.825606 & 0.000346157 & 0.00712697 & 0.00390571 \\ \hline
  1 & 0.543652 & 0.826861 & 0.001639    & 0.00743372 & 0.00917749 \\ \hline
  2 & 0.543134 & 0.826478 & 0.0018595   & 0.00760631 & 0.00949901 \\ \hline
  3 & 0.542945 & 0.826066 & 0.00184444  & 0.00805647 & 0.00940097 \\ \hline
  4 & 0.542819 & 0.825782 & 0.00184745  & 0.00829533 & 0.00932733 \\ \hline
  5 & 0.542737 & 0.825585 & 0.00182392  & 0.00845368 & 0.00926948 \\ \hline
  6 & 0.542679 & 0.82545  & 0.00180174  & 0.00856284 & 0.00923304 \\ \hline
  7 & 0.542649 & 0.825361 & 0.00178338  & 0.00867467 & 0.0091979  \\ \hline
  8 & 0.542627 & 0.82531  & 0.00177265  & 0.00873883 & 0.00918402 \\ \hline
  9 & 0.54261  & 0.825267 & 0.0017657   & 0.00876449 & 0.00916986 \\ \hline
 10 & 0.542599 & 0.82524  & 0.00176062  & 0.00877928 & 0.00916237 \\ \hline
 11 & 0.542593 & 0.825221 & 0.00175502  & 0.0088119  & 0.00915468 \\ \hline
 12 & 0.542589 & 0.825209 & 0.00175165  & 0.00882927 & 0.

In [36]:
test1(ar_coeff, 2, 3, 30)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.542276 & 0.824854 & -0.00427251 & 0.00794151 & 0.00924549 \\ \hline
  1 & 0.537501 & 0.824954 & -0.00480457 & 0.0119919  & 0.0151137  \\ \hline
  2 & 0.535147 & 0.820698 & -0.00503237 & 0.0148489  & 0.0146038  \\ \hline
  3 & 0.531651 & 0.814353 & -0.00519431 & 0.0186913  & 0.0138165  \\ \hline
  4 & 0.530575 & 0.81291  & -0.00517229 & 0.0193802  & 0.0139807  \\ \hline
  5 & 0.529662 & 0.811709 & -0.00523997 & 0.0195407  & 0.01414    \\ \hline
  6 & 0.528783 & 0.810541 & -0.0053174  & 0.0192929  & 0.0142839  \\ \hline
  7 & 0.527911 & 0.809466 & -0.00533842 & 0.0198213  & 0.0144949  \\ \hline
  8 & 0.527742 & 0.809198 & -0.00535016 & 0.0199383  & 0.0144879  \\ \hline
  9 & 0.527585 & 0.808978 & -0.00534118 & 0.0199366  & 0.0145046  \\ \hline
 10 & 0.527324 & 0.808677 & -0.00534107 & 0.0200464  & 0.0145841  \\ \hline
 11 & 0.527203 & 0.808513 & -0.00534378 & 0.0200546  & 0.0146016  \\ \hline
 12 & 0.527178 & 0.808464 & -0.00534166 & 0.0200529  & 0.

In [40]:
test1(ar_coeff, 2, 3, 25, 30)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.554591 & 0.813418 & -0.00355024 & 0.00709497 & 0.0151365  \\ \hline
  1 & 0.548439 & 0.813866 & -0.00255082 & 0.0123327  & 0.0072028  \\ \hline
  2 & 0.545398 & 0.810244 & -0.00297596 & 0.0185755  & 0.00646113 \\ \hline
  3 & 0.543592 & 0.807711 & -0.00325046 & 0.0196078  & 0.00633567 \\ \hline
  4 & 0.542475 & 0.805749 & -0.00327891 & 0.0211374  & 0.00658828 \\ \hline
  5 & 0.541648 & 0.80415  & -0.00316726 & 0.022678   & 0.0068988  \\ \hline
  6 & 0.541142 & 0.803131 & -0.00308922 & 0.0232613  & 0.00712411 \\ \hline
  7 & 0.540757 & 0.802646 & -0.00295353 & 0.0225897  & 0.00705066 \\ \hline
  8 & 0.54058  & 0.802324 & -0.00286452 & 0.0224894  & 0.00710056 \\ \hline
  9 & 0.540508 & 0.802121 & -0.00278827 & 0.0225944  & 0.00718182 \\ \hline
 10 & 0.540427 & 0.801947 & -0.00274743 & 0.0225086  & 0.00722666 \\ \hline
 11 & 0.540369 & 0.801846 & -0.00272796 & 0.0225049  & 0.0072403  \\ \hline
 12 & 0.540322 & 0.801765 & -0.00270709 & 0.0224889  & 0.

In [34]:
test1(ar_coeff, 2, 3, 10)

\begin{tabular}{rrrrrr}
\hline
  0 & 0.497919 & 0.558648 & 0.0173635  & 0.0302184 & 0.224626 \\ \hline
  1 & 0.393234 & 0.433367 & 0.00631743 & 0.0452213 & 0.240724 \\ \hline
  2 & 0.348566 & 0.273022 & 0.0226256  & 0.112343  & 0.610029 \\ \hline
  3 & 0.329335 & 0.256345 & 0.0350648  & 0.122695  & 0.618064 \\ \hline
  4 & 0.30932  & 0.227451 & 0.0326091  & 0.151573  & 0.693274 \\ \hline
  5 & 0.299517 & 0.206283 & 0.0369062  & 0.172283  & 0.785306 \\ \hline
  6 & 0.293857 & 0.202238 & 0.0363789  & 0.173997  & 0.786363 \\ \hline
  7 & 0.288783 & 0.198741 & 0.0368015  & 0.176434  & 0.786397 \\ \hline
  8 & 0.28538  & 0.193013 & 0.0355556  & 0.156669  & 0.811886 \\ \hline
  9 & 0.283064 & 0.190611 & 0.036306   & 0.157566  & 0.81837  \\ \hline
 10 & 0.281843 & 0.189423 & 0.0364975  & 0.158224  & 0.821237 \\ \hline
 11 & 0.280901 & 0.188544 & 0.0366514  & 0.158729  & 0.823176 \\ \hline
 12 & 0.280289 & 0.188029 & 0.0366627  & 0.159108  & 0.823999 \\ \hline
 13 & 0.279981 & 0.187674 & 0.036

## traverse parameter \phi

In [44]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def test1(ar_coeff, a1, a2, n):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 10
    # S = 12
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    # n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        # hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        # hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))

    return g
    # print(index_array)

    # array = np.array(g)
    # table = tabulate(array, tablefmt='latex_raw')
    
    # # Split the table into lines
    # lines = table.split('\n')
    
    # # Insert \hline after each row
    # latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    # print(latex_table)

    # EDR = EDR / n
    # L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)

    # print(tabulate(array, tablefmt='latex'))

    # print(g)
    # print(L)
    # print(EDR[0])
    # print(edr_est)

# Example usage
S = 10
nn = 10
ar_coeff = [0.2, 0.5, 0.5, 0.8]
g = [np.zeros((1, 1)) for _ in range(S)]
for i in range(S):
    g[i] = test1(ar_coeff, 2, 3, nn)[i]
index_array = list(range(len(g)))
for i in range(S):
    g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))
# for n in range(nn, nn+800, 100):
#     h = test1(ar_coeff, 2, 3, nn)
#     for i in range(S):
#         g[i] = np.vstack((g[i], h[i].reshape(-1,1)))

formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
# array = np.array(g)
table = tabulate(formatted_array, tablefmt='latex_raw')

# Split the table into lines
lines = table.split('\n')

# Insert \hline after each row
latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])

print(latex_table)

\begin{tabular}{rr}
\hline
 0 & 0.097  \\ \hline
 1 & 0.0905 \\ \hline
 2 & 0.5558 \\ \hline
 3 & 0.1525 \\ \hline
 4 & 0.6686 \\ \hline
 5 & 0.1264 \\ \hline
 6 & 0.1083 \\ \hline
 7 & 0.2009 \\ \hline
 8 & 0.1319 \\ \hline
 9 & 0.3784 \\ \hline
\hline \hline
\end{tabular} \hline


## traverse H

In [25]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test1(ar_coeff, a1, a2, H):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 10
    # S = 12
    noise = np.zeros((num_N, n_obs+S))
    # H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 10
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        # hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        # hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))

    return g
    # print(index_array)

    # array = np.array(g)
    # table = tabulate(array, tablefmt='latex_raw')
    
    # # Split the table into lines
    # lines = table.split('\n')
    
    # # Insert \hline after each row
    # latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    # print(latex_table)

    # EDR = EDR / n
    # L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)

    # print(tabulate(array, tablefmt='latex'))

    # print(g)
    # print(L)
    # print(EDR[0])
    # print(edr_est)

# Example usage
S = 10
nn = 5
ar_coeff = [0.2, 0.5, 0.5, 0.8]
g = [np.zeros((1, 1)) for _ in range(S)]
for i in range(S):
    g[i] = test1(ar_coeff, 2, 3, nn)[i]
index_array = list(range(len(g)))
for i in range(S):
    g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))
for n in range(nn, nn+60, 10):
    h = test1(ar_coeff, 2, 3, nn)
    for i in range(S):
        g[i] = np.vstack((g[i], h[i].reshape(-1,1)))

formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
# array = np.array(g)
table = tabulate(formatted_array, tablefmt='latex_raw')

# Split the table into lines
lines = table.split('\n')

# Insert \hline after each row
latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])

print(latex_table)

\begin{tabular}{rrrrrrrr}
\hline
 0 & 1.7292 & 0.1801 & 0.0318 & 0.3015 & 1.9715 & 0.0913 & 0.0048 \\ \hline
 1 & 0.6985 & 0.0845 & 0.0398 & 0.0028 & 0.7006 & 0.1725 & 0.21   \\ \hline
 2 & 1.4618 & 0.0128 & 1.1889 & 0.3274 & 1.7366 & 0.171  & 0.2446 \\ \hline
 3 & 0.0616 & 0.0314 & 2.4018 & 0.1387 & 1.6995 & 0.4692 & 0.2646 \\ \hline
 4 & 0.3632 & 0.0545 & 3.0432 & 0.0731 & 7.4903 & 4.6551 & 0.2447 \\ \hline
 5 & 0.0066 & 0.0765 & 0.908  & 0.0638 & 7.5635 & 3.6308 & 0.2508 \\ \hline
 6 & 0.2621 & 0.1159 & 2.9838 & 0.0571 & 8.3918 & 0.7494 & 0.2569 \\ \hline
 7 & 0.0533 & 0.0133 & 2.8049 & 0.0466 & 8.3823 & 0.8254 & 0.2621 \\ \hline
 8 & 0.4367 & 0.0163 & 2.7354 & 0.0375 & 8.2752 & 2.5693 & 0.2642 \\ \hline
 9 & 0.0101 & 0.0204 & 2.6399 & 0.0285 & 8.2319 & 2.5099 & 0.2679 \\ \hline
\hline \hline
\end{tabular} \hline


## traverse number of replicas

In [46]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test1(ar_coeff, a1, a2, n):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 20
    # S = 12
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    # n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        # hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        # hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))

    return g
    # print(index_array)

    # array = np.array(g)
    # table = tabulate(array, tablefmt='latex_raw')
    
    # # Split the table into lines
    # lines = table.split('\n')
    
    # # Insert \hline after each row
    # latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    # print(latex_table)

    # EDR = EDR / n
    # L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)

    # print(tabulate(array, tablefmt='latex'))

    # print(g)
    # print(L)
    # print(EDR[0])
    # print(edr_est)

# Example usage
S = 10
nn = 50
ar_coeff = [0.2, 0.5, 0.5, 0.8]
g = [np.zeros((1, 1)) for _ in range(S)]
for i in range(S):
    g[i] = test1(ar_coeff, 2, 3, nn)[i]
index_array = list(range(len(g)))
for i in range(S):
    g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))
for n in range(nn, nn+80, 10):
    h = test1(ar_coeff, 2, 3, nn)
    for i in range(S):
        g[i] = np.vstack((g[i], h[i].reshape(-1,1)))

formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
# array = np.array(g)
table = tabulate(formatted_array, tablefmt='latex_raw')

# Split the table into lines
lines = table.split('\n')

# Insert \hline after each row
latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])

print(latex_table)

\begin{tabular}{rrrrrrrrrr}
\hline
 0 & 0.011  & 0.0098 & 0.0148 & 0.0079 & 0.0293 & 0.0096 & 0.0125 & 0.0125 & 0.0283 \\ \hline
 1 & 0.0449 & 0.061  & 0.0762 & 0.0551 & 0.0811 & 0.0416 & 0.0636 & 0.0505 & 0.0861 \\ \hline
 2 & 0.0704 & 0.0694 & 0.0862 & 0.0659 & 0.094  & 0.0526 & 0.0718 & 0.06   & 0.0975 \\ \hline
 3 & 0.0894 & 0.0239 & 0.088  & 0.0686 & 0.0971 & 0.0531 & 0.0559 & 0.0617 & 0.0988 \\ \hline
 4 & 0.0788 & 0.0274 & 0.0881 & 0.0694 & 0.0975 & 0.0517 & 0.0572 & 0.0649 & 0.0984 \\ \hline
 5 & 0.1161 & 0.0273 & 0.0885 & 0.0702 & 0.0982 & 0.0498 & 0.0577 & 0.0662 & 0.098  \\ \hline
 6 & 0.0589 & 0.0263 & 0.0885 & 0.0711 & 0.0979 & 0.0496 & 0.0584 & 0.0677 & 0.0978 \\ \hline
 7 & 0.0695 & 0.0255 & 0.0884 & 0.0713 & 0.0975 & 0.0498 & 0.0583 & 0.0689 & 0.0977 \\ \hline
 8 & 0.0702 & 0.0255 & 0.0885 & 0.0714 & 0.097  & 0.0499 & 0.0586 & 0.0698 & 0.0975 \\ \hline
 9 & 0.0664 & 0.0252 & 0.0885 & 0.0715 & 0.0971 & 0.0499 & 0.0587 & 0.0698 & 0.0974 \\ \hline
\hline \hline
\end{tabula

## traverse number of samples

In [48]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test1(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    # S = 12
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 10
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] ,_ , M, Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum((phi[j] ** a) * (Q2[a] @ V1[a] @ np.linalg.inv(Q2[a]))[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1
    
    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        # hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        # hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))

    return g
    # print(index_array)

    # array = np.array(g)
    # table = tabulate(array, tablefmt='latex_raw')
    
    # # Split the table into lines
    # lines = table.split('\n')
    
    # # Insert \hline after each row
    # latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    # print(latex_table)

    # EDR = EDR / n
    # L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)

    # print(tabulate(array, tablefmt='latex'))

    # print(g)
    # print(L)
    # print(EDR[0])
    # print(edr_est)

# Example usage
S = 10
nn = 100
ar_coeff = [0.2, 0.5, 0.5, 0.8]
g = [np.zeros((1, 1)) for _ in range(S)]
for i in range(S):
    g[i] = test1(ar_coeff, 2, 3, nn)[i]
index_array = list(range(len(g)))
for i in range(S):
    g[i] = np.vstack((np.array([[index_array[i]]]) ,g[i]))
for n in range(nn, nn+800, 100):
    h = test1(ar_coeff, 2, 3, nn)
    for i in range(S):
        g[i] = np.vstack((g[i], h[i].reshape(-1,1)))

formatted_array = np.vectorize(lambda x: f"{x:.4f}")(g)
# array = np.array(g)
table = tabulate(formatted_array, tablefmt='latex_raw')

# Split the table into lines
lines = table.split('\n')

# Insert \hline after each row
latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])

print(latex_table)

\begin{tabular}{rrrrrrrrrr}
\hline
 0 & 0.0013 & 0.0043 & 0.0054 & 0.017  & 0.0366 & 0.0091 & 0.0136 & 0.024  & 0.0076 \\ \hline
 1 & 0.0264 & 0.0332 & 0.0274 & 0.0525 & 0.0002 & 0.0226 & 0.0415 & 0.0112 & 0.0275 \\ \hline
 2 & 0.0249 & 0.0362 & 0.0306 & 0.0563 & 0.0041 & 0.0247 & 0.0447 & 0.0157 & 0.0318 \\ \hline
 3 & 0.0711 & 0.0368 & 0.0314 & 0.0569 & 0.0049 & 0.0252 & 0.0454 & 0.0162 & 0.0325 \\ \hline
 4 & 0.0847 & 0.037  & 0.0316 & 0.057  & 0.0051 & 0.0253 & 0.0455 & 0.0163 & 0.0327 \\ \hline
 5 & 0.0572 & 0.037  & 0.0317 & 0.057  & 0.0052 & 0.0254 & 0.0456 & 0.0163 & 0.0327 \\ \hline
 6 & 0.064  & 0.037  & 0.0317 & 0.057  & 0.0052 & 0.0254 & 0.0456 & 0.0163 & 0.0327 \\ \hline
 7 & 0.0946 & 0.037  & 0.0318 & 0.057  & 0.0052 & 0.0254 & 0.0456 & 0.0163 & 0.0327 \\ \hline
 8 & 0.0196 & 0.037  & 0.0318 & 0.057  & 0.0052 & 0.0254 & 0.0456 & 0.0163 & 0.0327 \\ \hline
 9 & 0.0066 & 0.037  & 0.0318 & 0.057  & 0.0052 & 0.0254 & 0.0456 & 0.0163 & 0.0327 \\ \hline
\hline \hline
\end{tabula

# Take sample covariance matrix by unwhitened X

In [105]:
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
def test2(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    # n_obs = 10
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 100
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] , M , _ , Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                edr[0] = np.linalg.inv(np.cov(X.T)) @ edr[0]
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = np.linalg.inv(np.cov(X.T)) @ K_largest_eigenvectors 
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            # hat[q - 1] += edr_est
            hat[q - 1] += np.real(edr_est)
        n1 += 1

    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')

    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)
    # print(tabulate(array, tablefmt='latex'))
    # print(g)
    print(L)
    print(EDR[0])
    print(edr_est)
# Example usage
ar_coeff = [0.2, 0.3, 0.5, 0.8]
test2(ar_coeff, 2, 3, 40)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.551862 & 0.804171 & 0.00834409 & -0.0261078 & 0.0195826  \\ \hline
 1 & 0.539692 & 0.795889 & 0.00613204 & -0.0412635 & 0.0114321  \\ \hline
 2 & 0.530437 & 0.786094 & 0.0053654  & -0.0479281 & 0.00810836 \\ \hline
 3 & 0.519903 & 0.758112 & 0.00749389 & -0.0371327 & 0.0191199  \\ \hline
 4 & 0.515066 & 0.752194 & 0.00796879 & -0.0370938 & 0.0180847  \\ \hline
 5 & 0.51197  & 0.737299 & 0.00777171 & -0.0528522 & 0.0277184  \\ \hline
 6 & 0.509376 & 0.734327 & 0.00798675 & -0.0533566 & 0.0269965  \\ \hline
 7 & 0.507718 & 0.732136 & 0.00810177 & -0.0544358 & 0.0268082  \\ \hline
 8 & 0.5069   & 0.730752 & 0.00796889 & -0.0558828 & 0.0270028  \\ \hline
 9 & 0.506369 & 0.729526 & 0.00773089 & -0.0576772 & 0.0274406  \\ \hline
\hline \hline
\end{tabular} \hline
0.07832799413471236
[ 0.5708818   0.76628979 -0.00510866 -0.00327897]
[[ 0.46083377]
 [ 0.83782069]
 [ 0.18676375]
 [-0.22540639]]


# By multiplying uniform Q

In [4]:
#from 0 to Q double sum
#2 * ar_series[0][t]/(5 * ar_series[1][t] + 1)
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler

def test3(ar_coeff, a1, a2, n_obs):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    S = 10
    noise = np.zeros((num_N, n_obs+S))
    H = 5
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    n = 40
    edr = np.zeros((S+1, P))
    EDR = np.zeros((1, P))
    while n1 < n:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            edr[a] , M , _ , Q1 = sir_modified1(X, y[a], H, K)
            if a == 0:
                if edr[0][0]<0:
                    edr[0] = -edr[0]
                edr[0] = edr[0]/np.linalg.norm(edr[0])
                EDR += edr[0]
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, q))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # # K_largest_eigenvectors = edr_est[:, K_index]
            # K_largest_eigenvectors = eigenvectors1[:, K_index]
            K_largest_eigenvectors = edr_est[:, K_index]
            #multiply R after that
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += np.real(edr_est)
        n1 += 1

    index_array = list(range(len(hat)))
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = abs(hat[i][0] / hat[i][1] - a1/a2)
        hat[i] = np.vstack((hat[i], g[i].reshape(1,-1)))
        hat[i] = np.vstack((np.array([[index_array[i]]]), hat[i]))
    
    # print(index_array)
    array = np.array(hat)
    table = tabulate(array, tablefmt='latex_raw')

    # Split the table into lines
    lines = table.split('\n')
    
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    
    print(latex_table)
    EDR = EDR / n
    L = abs(EDR[0][0] / EDR[0][1] - a1/a2)
    # array = np.array(hat)
    # print(tabulate(array, tablefmt='latex'))
    # print(g)
    print(L)
    print(EDR[0])
    print(edr_est)
# Example usage
ar_coeff = [0.2, 0.5, 0.5, 0.8]
test3(ar_coeff, 3, 7, 1000)

\begin{tabular}{rrrrrr}
\hline
 0 & 0.351463 & -0.0469681 & -0.00647294 & -0.00410173 & 7.9116  \\ \hline
 1 & 0.336751 & -0.0474621 & -0.00683192 & -0.0050314  & 7.52373 \\ \hline
 2 & 0.335613 & -0.0474905 & -0.00683665 & -0.00525159 & 7.49551 \\ \hline
 3 & 0.335512 & -0.0474907 & -0.00684354 & -0.00534694 & 7.49337 \\ \hline
 4 & 0.335501 & -0.0474909 & -0.00684341 & -0.00541173 & 7.49309 \\ \hline
 5 & 0.335498 & -0.0474914 & -0.00684392 & -0.00543649 & 7.49297 \\ \hline
 6 & 0.335497 & -0.047492  & -0.00684429 & -0.00544617 & 7.49285 \\ \hline
 7 & 0.335496 & -0.0474925 & -0.00684415 & -0.00545523 & 7.49276 \\ \hline
 8 & 0.335496 & -0.047493  & -0.00684432 & -0.00545868 & 7.49268 \\ \hline
 9 & 0.335496 & -0.0474932 & -0.00684451 & -0.00546024 & 7.49264 \\ \hline
\hline \hline
\end{tabular} \hline
0.0008008700502919464
[3.93228047e-01 9.19249909e-01 1.04862632e-04 2.88912916e-03]
[[ 0.33155318]
 [ 0.94220931]
 [-0.02378119]
 [-0.04181567]]


In [ ]:
X1 = X - np.mean(X, axis=0)

# Checking if the covariance relationship exists

In [9]:
import numpy as np
from tabulate import tabulate
ar_coeff = [0.2, 0.5, 0.5, 0.8]
num_N = 5
n_obs = 10
S = 10
noise = np.zeros((num_N, n_obs+S))
H = 5
P = 4
K = 1
y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
hat = [np.zeros((P, 1)) for _ in range(S)]
g = np.zeros((S, 1))
n1 = 0
l = 1  
n = 100
edr = np.zeros((S+1, P))
EDR = np.zeros((1, P))

for h in range(num_N):
    noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
ar_series = np.zeros((num_N, n_obs+S))
for t in range(0, n_obs+S):
    ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
    ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
    ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
    ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
    ar_series[4][t] = 2 * ar_series[0][t] + 3 * ar_series[1][t] + noise[4][t]
for a in range(0, S+1):
    y[a] = ar_series[4][a:n_obs+a]
X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)

X1 = X - np.mean(X, axis=0)
#reduced QR decomposition
Q, R = np.linalg.qr(X1)

R @ R

array([[ 9.14440926,  7.05865946, -0.43096876, -2.5844906 ],
       [ 0.        , 14.03190128, -1.5150993 , -9.67387158],
       [ 0.        ,  0.        ,  1.74528559, -0.79628651],
       [ 0.        ,  0.        ,  0.        ,  5.52303073]])

In [10]:
(R @ R).shape

(4, 4)

In [33]:
np.cov(X.T).shape 

(4, 4)

In [12]:
np.linalg.inv(R)

array([[-0.33069085,  0.09204591,  0.01396628,  0.03964654],
       [-0.        , -0.26695726, -0.06042237, -0.18704795],
       [-0.        , -0.        , -0.75694922, -0.06986162],
       [-0.        , -0.        , -0.        , -0.42551147]])

In [13]:
R.T

array([[-3.02397243,  0.        ,  0.        ,  0.        ],
       [-1.04265491, -3.74591795,  0.        ,  0.        ],
       [ 0.02743387,  0.29901246, -1.32109257,  0.        ],
       [ 0.17207493,  1.5975522 ,  0.21690056, -2.35011292]])

In [15]:
R.T @ R

array([[ 9.14440926,  3.15295971, -0.08295927, -0.52034985],
       [ 3.15295971, 15.11903055, -1.14868021, -6.16371425],
       [-0.08295927, -1.14868021,  1.83544666,  0.19586299],
       [-0.52034985, -6.16371425,  0.19586299,  8.15185941]])

In [16]:
R.T @ np.linalg.inv(R)

array([[ 1.        , -0.2783443 , -0.04223364, -0.11989003],
       [ 0.34479643,  0.90402788,  0.21177524,  0.65932863],
       [-0.00907213, -0.07729837,  0.98231611,  0.03745166],
       [-0.0569036 , -0.41063937, -0.25830736,  0.69285028]])

In [17]:
np.linalg.inv(R) @ R

array([[ 1.00000000e+00,  0.00000000e+00,  3.46944695e-18,
         2.77555756e-17],
       [ 0.00000000e+00,  1.00000000e+00, -1.38777878e-17,
        -5.55111512e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -2.77555756e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [18]:
R @ R/1000

array([[ 0.00914441,  0.00705866, -0.00043097, -0.00258449],
       [ 0.        ,  0.0140319 , -0.0015151 , -0.00967387],
       [ 0.        ,  0.        ,  0.00174529, -0.00079629],
       [ 0.        ,  0.        ,  0.        ,  0.00552303]])

In [19]:
R.T @ R

array([[ 9.14440926,  3.15295971, -0.08295927, -0.52034985],
       [ 3.15295971, 15.11903055, -1.14868021, -6.16371425],
       [-0.08295927, -1.14868021,  1.83544666,  0.19586299],
       [-0.52034985, -6.16371425,  0.19586299,  8.15185941]])

In [ ]:
import itertools

# Define the step size and the range
step_size = 1
range_values = [round(i * step_size, 1) for i in range(int(1/step_size) + 1)]

# Generate all possible combinations
grid_points = list(itertools.product(range_values, repeat=4))

# Display the result
for point in grid_points:
    test2(point)